In [1]:
import numpy as np
from keras.initializers import glorot_uniform

In [2]:
class GCNLayer():
    def __init__(self, nInputs, nOutputs, activation=None, name=''):
        self.nInputs = nInputs
        self.nOutputs = nOutputs
        self.W = glorot_uniform(self.nOutputs, self.nInputs)
        self.activation = activation
        self.name = name

    def forward(self, A, X, W=None):
        self._X = (A @ X).T # message passing

        if W is None:
            W = self.W
        
        H = W @ self._X # net input
        if self.activation != None:
            H = self.activation(H) 
        self._H = H
        return self._H.T

    def backward(self, optim, update=True):
        dtanh = 1 - np.asarray(self._H.T)**2
        d2 = np.multiply(optim.out, dtanh)

        optim.out = d2 @ self.W

        dW = np.asanyarray(d2.T @ self._X.T) / optim.bs
        dwDecay = self.W * optim.wd / optim.bs

        if update:
            self.W -= optim.lr * (dW + dwDecay)
            return dW + dwDecay

class GradientDescentOptim():
    def __init__(self, lr, wDecay):
        self.lr = lr
        self.wDecay = wDecay
        self._yHat = None
        self._y = None
        self._out = None
        self.bs = None
        self.trainNodes = None

    def __call__(self, yHat, y, trainNodes=None):
        self.y = y
        self.yHat = yHat

        if trainNodes != None:
            self.trainNodes = trainNodes
        else:
            self.trainNodes = np.arange(yHat.shape[0])

        self.bs = self.trainNodes.shape[0]

    @property
    def out(self):
        return self._out
    
    @out.setter
    def out(self, y):
        self._out = y

class SoftmaxLayer():
    def __init__(self, nInputs, nOutputs, name=''):
        self.nInputs = nInputs
        self.nOutputs = nOutputs
        self.W = glorot_uniform(self.nOutputs, self.nInputs)
        self.b = np.zeros((self.nOutputs, 1))
        self.name = name
        self._X = None

    def shift(self, proj):
        shiftX = proj - np.max(proj, axis=0, keepdims=True)
        exps = np.exp(shiftX)
        return exps / np.sum(exps, axis=0, keepdims=True)

    def forward(self, X, W=None, b=None):
        self._X = X.T
        if W is None:
            W = self.W
        if b is None:
            b = self.b
        
        proj = np.asarray(W @ self._X) + b
        return self.shift(proj).T

    def backward(self, optim, update=True):
        trainMask = np.zeros(optim.yHat.shape[0])
        trainMask[optim.trainNodes] = 1
        trainMask = trainMask.reshape((-1, 1))

        d1 = np.asarray((optim.yHat - optim.y))
        d1 = np.multiply(d1, trainMask)

        optim.out = d1 @ self.W
        dW = (d1.T @ self._X.T) / optim.bs
        db = d1.T.sum(axis=1, keepdims=True) / optim.bs

        dWDecay = self.W * optim.wDecay / optim.bs

        if update:
            self.W -= optim.lr * (dW + dWDecay)
            self.b -= optim.lr * db.reshape(self.b.shape)

        return dW + dWDecay, db.reshape(self.b.shape)

In [ ]:
class GCNN():
    def __init__(self, nInputs, noutputs, nLayers, hiddenSizes, activation, seed=0):
        self.nInputs = nInputs
        self.nOutputs = noutputs
        self.nLayers = nLayers
        self.hiddenSizes = hiddenSizes

        np.random.seed(seed)

        self.layers = []

        # Add input layer
        self.layers.append(GCNLayer(nInputs, hiddenSizes[0], activation, name='in'))

        # Add hidden layer
        for layer in range(nLayers):
            self.layers.append(GCNLayer(self.layers[-1].W.shape[0], hiddenSizes[layer], activation, name="hidden layer{}".format(layer+1)))

        # Output layer
        self.layers.append(SoftmaxLayer(hiddenSizes[-1], noutputs, activation, name='out'))

    def embedding(self, A, X):
        H = X
        for layer in self.layers[:-1]:
            H = layer.forward(A, H)
        return np.asarray(H)

    def forward(self, A, X):
        H = self.embedding(A, X)

        p = self.layers[-1].forward(H)

        return np.asarray(p)  


    def backward(self, optim, update=True):
        self.layers[-1].backward(optim)
        for layer in reversed(self.layers):
            H = layer.backward(optim)